# Text generation using tensor2tensor on Cloud AI Platform

This notebook illustrates the use of [tensor2tensor](https://github.com/tensorflow/tensor2tensor) library to do from-scratch, distributed training of a training model. Then, the trained model is used to complete new poems.

## Install tensor2tensor, and specify Google Cloud Platform project and bucket

Install the necessary packages. tensor2tensor will give us the Transformer model. Project Gutenberg gives us access to historical poems.

**p.s.** Note that this notebook uses Python 2 because Project Gutenberg relies on BSD-DB which was deprecated in Python 3 and removed from the standard library. tensor2tensor itself can be used on Python 3.

In [ ]:
%%bash
pip freeze | grep tensor

In [ ]:
%%bash
pip install tensor2tensor=1.13.1 tensorflow=1.13.1 tensorflow-serving-api=1.13 gutenberg
pip install tensorflow_hub

# install from sou
#git clone https://github.com/tensorflow/tensor2tensor.git
#cd tensor2tensor
#yes | pip install --user -e .

In [ ]:
%%bash
pip freeze | grep tensor

In [ ]:
import os
PROJECT = "cloud-training-demos" # Replace with your Project ID
BUCKET = "cloud-training-demos-ml" # Replace with your bucket name
REGION = "us-central1" # Replace with your bucket region

# this is what this notebook is demonstrating
PROBLEM = "poetry_line_problem"

# for bash
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["PROBLEM"] = PROBLEM

# os.environ["PATH"] = os.environ["PATH"] + ":" + os.getcwd() + "/tensor2tensor/tensor2tensor/bin"

In [ ]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## Download data

We will get some [poetry anthologies](https://www.gutenberg.org/wiki/Poetry_(Bookshelf)) from Project Gutenberg.

In [ ]:
%%bash
rm -rf data/poetry
mkdir -p data/poetry

In [ ]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
import re

books = [
    # bookid, skip N lines, title
    (26715, 1000, "Victorian songs"),
    (30235, 580, "Baldwin collection"),
    (35402, 710, "Swinburne collection"),
    (574, 15, "Blake"),
    (1304, 172, "Bulchevys collection"),
    (19221, 223, "Palgrave-Pearse collection"),
    (15553, 522, "Knowles collection")
]

with open("data/poetry/raw.txt", "w") as ofp:
    lineno = 0
    for (id_nr, toskip, title) in books:
        startline = lineno
        text = strip_headers(load_etext(id_nr)).strip()
        lines = text.split("\n")[toskip:]
        # any line that is all upper case is a title or author name
        # also don't want any lines with years (numbers)
        for line in lines:
            if (len(line) > 0
                and line.upper() != line
                and not re.match(".*[0-9]+.*", line)
                and len(line) < 50
                ):
                cleaned = re.sub("[^a-z\"\-]+", " ", line.strip().lower())
                ofp.write(cleaned)
                ofp.write("\n")
                lineno += 1
            else:
                ofp.write("\n")
        print("Wrote lines {} to {} from {}".format(startline, lineno, title))

In [ ]:
!wc -l data/poetry/*.txt

## Create training dataset

We are going to train an ML model to write poetry given a starting point. We'll give it one line, and it's going to tell us the next line. So, naturally, we will train it on real poetry. Our feature will be a line of a poem and the label will be next line of that poem.

Our training dataset will consist of two files. The first file will consist of the input lines of poetry and the other file will consist of the corresponding output lines, one output line per input line.

In [ ]:
with open("data/poetry/raw.txt", "r") as rawfp,\
  open("data/poetry/input.txt", "w") as infp,\
  open("data/poetry/output.txt", "w") as outfp:
    
    prev_line = ""
    for curr_line in rawfp:
        curr_line = curr_line.strip()
        # poems break at empty lines, so this ensures we train only on lines of the same poem
        if len(prev_line) > 0 and len(curr_line) > 0:
            infp.write(prev_line + "\n")
            outfp.write(curr_line + "\n")
        prev_line = curr_line

In [ ]:
!head -5 data/poetry/*.txt

We do not need to generate the data beforehand $-$ instead, we can have tensor2tensor create the training dataset for us. So, in the code below, we'll use only `data/poetry/raw.txt` $-$ obviously, this allows us to productionise our model better. Simply keep collecting raw data and generate the training/test data at the time of training.

## Set up problem

The Problem in tensor2tensor is where you specify parameters like the size of your vocabulary and where to get the training data from.

In [ ]:
%%bash
rm -rf poetry
mkdir -p poetry/trainer

In [ ]:
%%writefile poetry/trainer/problem.py
import os
import tensorflow as tf
from tensor2tensor.utils import registry
from tensor2tensor.models import transformer
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import generator_utils

tf.summary.FileWriteCache.clear() # ensure filewriter cache is clear for TensorBoard events file

@registry.register_problem
class PoetryLineProblem(text_problems.Text2TextProblem):
    """Predict next line of poetry from the last line from Gutenberg texts.
    """
    
    @property
    def approx_vocab_size(self):
        return 2**13 # ~8k
    
    @property
    def is_generate_per_split(self):
        # generate data will NOT shard the data into TRAIN and EVAL for us
        return False
    
    @property
    def dataset_splits(self):
        """Splits of data to produce and number of output shards for each.
        """
        # 10% evaluation data
        return [{
            "split": problem.DatasetSplit.TRAIN,
            "shards": 90,
        }, {
            "split": problem.DatasetSplit.Eval,
            "shards": 10
        }]
    
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        with open("data/poetry/raw.txt", "r") as rawfp:
            prev_line = ""
            for curr_line in rawfp:
                curr_line = curr_line.strip()
            # poems break at empty lines, so this ensures we train only on lines of the same poem
            if len(prev_line) > 0 and len(curr_line) > 0:
                yield {
                    "inputs": prev_line,
                    "targets": curr_line
                }
            prev_line = curr_line
            
# Smaller than the typical translate model, and with more regularisation
@registry.register_hparams
def transformer_poetry():
    hparams = transformer.transformer_base()
    hparams.num_hidden_layers = 2
    hparams.hidden_size = 128
    hparams.filter_size = 512
    hparams.num_heads = 4
    hparams.attention_dropout = 0.6
    hparams.layer_prepostprocess_dropout = 0.6
    hparams.learning_rate = 0.05
    return hparams

@registry.register_hparams
def transformer_poetry_tpu():
    hparams = transformer_poetry()
    transformer.update_hparams_for_tpu(hparams)
    return hparams

# hyperparameter tuning ranges
@registry.register_ranged_hparams
def transformer_poetry_range(rhp):
    rhp.set_float("learning_rate", 0.05, 0.25, scale=rhp.LOG_SCALE)
    rhp.set_int("num_hidden_layers", 2, 4)
    rhp.set_discrete("hidden_size", [128, 256, 512])
    rhp.set_float("attention_dropout", 0.4, 0.7)

In [ ]:
%%writefile poetry/trainer/__init__.py
from . import problem

In [ ]:
%%writefile poetry/setup.py
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = [
    "tensor2tensor"
]

setup(
    name="poetry",
    version="0.1",
    author="Google",
    author_email="training-feedback@cloud.google.com",
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description="Poetry Line Problem",
    requires=[]
)

In [ ]:
!touch poetry/__init__.py

In [ ]:
!find poetry